In [1]:
import math
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


from IPython.core.debugger import set_trace

# Minimum Detectable Signal Calculator

http://www.ti.com/lit/an/slaa652/slaa652.pdf


In [10]:
def cascadeNoiseFigure(NF_dB=np.array([]), Gain_dB=np.array([])):
    NF_cum_dB = []
    if NF_dB.size != Gain_dB.size:
        return NF_cum_dB
    if NF_dB.size == 1:
        return NF_dB
    # Convert to linear
    G = np.power(10, Gain_dB/10)    
    F = np.power(10, NF_dB/10)
    Gc = np.cumprod(G)
    # First element
    F_cum = F[0]
    # set_trace()
    # Next Elements
    for n in np.arange(1,NF_dB.size):
        F_cum = F_cum + (F[n]-1)/Gc[n-1]
    NF_cum_dB = 10*np.log10(F_cum)
    return NF_cum_dB

def calculateSnrOut(Pin_dBm, Ta_K, AntGain_dB, BW_Hz, NF_dB=np.array([]), Gain_dB=np.array([])):
    k = 1.3806485279e-23
    NF2 = cascadeNoiseFigure(NF_dB, Gain_dB)
    Nin_dB = 10*np.log10(k * Ta_K * BW_Hz) + 30
    SNRin_dB = Pin_dBm + AntGain_dB - Nin_dB
    SNRout_dB = SNRin_dB - NF2
    return SNRout_dB


In [55]:
pin_dBm = -125
antTemp_K = 50
antGain_dB = 8
bw_Hz = 15000
NF = np.array([0.42, 2.8, 2.5, 5])
Gain = np.array([15, -2.8, 42, 60])

# calculateSnrOut(Pin_dBm=pin_dBm, Ta_K=antTemp_K, AntGain_dB=Gant_dB, BW_Hz=bw_Hz, NF_dB=NF, Gain_dB=Gain)

def f(Ta,BW):
    snr=calculateSnrOut(Pin_dBm=pin_dBm, Ta_K=Ta, AntGain_dB=antGain_dB, BW_Hz=BW, NF_dB=NF, Gain_dB=Gain)
    print("SNR out",snr,"dB")
    pout = pin_dBm + antGain_dB + np.cumsum(Gain)
    print("Pout",pout,"dBm")
    
interact(f, Ta=widgets.FloatSlider(min=5,max=300,step=1,value=50),
        BW=widgets.FloatSlider(min=1e3,max=200e3,step=1e3,value=100e3));


interactive(children=(FloatSlider(value=50.0, description='Ta', max=300.0, min=5.0, step=1.0), FloatSlider(val…